In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input, GRU, LSTM,BatchNormalization, MaxPool2D, GlobalMaxPool2D, TimeDistributed
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
import os
from tensorflow.keras import datasets, layers, models
import glob
import keras

Using TensorFlow backend.


In [2]:
# Detect hardware
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError: # If TPU not found
    tpu = None
    
# Select appropriate distribution strategy
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
else:
    strategy = tf.distribute.get_strategy() # Default strategy that works on CPU and single GPU
    print('Running on CPU instead')

print("Number of accelerators: ", strategy.num_replicas_in_sync)

Running on CPU instead
Number of accelerators:  1


In [3]:
path = '/kaggle/input/devnagri-hindi-dataset/DevanagariHandwrittenCharacterDataset'

In [4]:
train_datagen = ImageDataGenerator(
    rescale = 1./255, 
    horizontal_flip = False, 
    validation_split = 0.4
)


In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [6]:
train_set = train_datagen.flow_from_directory(path + '/Train', target_size = (64, 64), batch_size = 400, class_mode = 'categorical', subset='training')

Found 46920 images belonging to 46 classes.


In [7]:
validation_set = train_datagen.flow_from_directory(path + '/Train', target_size = (64, 64), batch_size = 400, class_mode = 'categorical', subset='validation')

Found 31280 images belonging to 46 classes.


In [8]:
test_set = test_datagen.flow_from_directory(path + '/Test', target_size = (64, 64), batch_size = 400, class_mode = 'categorical' )

Found 13800 images belonging to 46 classes.


In [9]:
Labels = train_set.class_indices
print(Labels)
num_classes = len(Labels)

{'character_10_yna': 0, 'character_11_taamatar': 1, 'character_12_thaa': 2, 'character_13_daa': 3, 'character_14_dhaa': 4, 'character_15_adna': 5, 'character_16_tabala': 6, 'character_17_tha': 7, 'character_18_da': 8, 'character_19_dha': 9, 'character_1_ka': 10, 'character_20_na': 11, 'character_21_pa': 12, 'character_22_pha': 13, 'character_23_ba': 14, 'character_24_bha': 15, 'character_25_ma': 16, 'character_26_yaw': 17, 'character_27_ra': 18, 'character_28_la': 19, 'character_29_waw': 20, 'character_2_kha': 21, 'character_30_motosaw': 22, 'character_31_petchiryakha': 23, 'character_32_patalosaw': 24, 'character_33_ha': 25, 'character_34_chhya': 26, 'character_35_tra': 27, 'character_36_gya': 28, 'character_3_ga': 29, 'character_4_gha': 30, 'character_5_kna': 31, 'character_6_cha': 32, 'character_7_chha': 33, 'character_8_ja': 34, 'character_9_jha': 35, 'digit_0': 36, 'digit_1': 37, 'digit_2': 38, 'digit_3': 39, 'digit_4': 40, 'digit_5': 41, 'digit_6': 42, 'digit_7': 43, 'digit_8': 4

<h3 style="color:purple;">Model with CNN is working fine</h3>

In [10]:
momentum = .9

cnn_model = Sequential()

cnn_model.add(Conv2D(64, (3,3), input_shape=(64, 64, 3),   padding='same', activation='relu'))

cnn_model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
cnn_model.add(BatchNormalization(momentum=momentum))

cnn_model.add(MaxPool2D())

cnn_model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
cnn_model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
cnn_model.add(BatchNormalization(momentum=momentum))

cnn_model.add(MaxPool2D())

cnn_model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
cnn_model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
cnn_model.add(BatchNormalization(momentum=momentum))

cnn_model.add(MaxPool2D())

cnn_model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
cnn_model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
cnn_model.add(BatchNormalization(momentum=momentum))

# flatten..
cnn_model.add(GlobalMaxPool2D())


cnn_model.add(Dense(num_classes, activation = 'softmax'))


# Display the architecture
cnn_model.summary()
cnn_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

history = cnn_model.fit(train_set, epochs=1, verbose=1, validation_data = validation_set, shuffle=True)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 128)      

<h3 style="color:purple;">#########################################################################################################################################################################################################################################################r</h3>

<h3 style="color:purple;">METHOD 1 <br><br> When I am trying to add the GRU layer with CNN then its throwing an input shape error</h3>

<h5 style="color:red;"> ValueError: Error when checking input: expected time_distributed_7_input to have 5 dimensions, but got array with shape (400, 64, 64, 3) </h5>

<h5 style="color:blue;">https://medium.com/smileinnovation/training-neural-network-with-image-sequence-an-example-with-video-as-input-c3407f7a0b0f</h5>

In [11]:
def build_convnet(shape=(64, 64, 3)):
    momentum = .9
    model = Sequential()
    model.add(Conv2D(64, (3,3), input_shape=shape,
        padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalMaxPool2D())
    return model



def gru_model(shape=(400,64, 64, 3), nbout=46):

    convnet = build_convnet()
    
    # then create our final model
    model = Sequential()

    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, adding GRU layer
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model



In [12]:
cnn_gru_model = gru_model()

# Display the architecture
cnn_gru_model.summary()
cnn_gru_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

history = cnn_gru_model.fit(train_set, epochs=2, verbose=1, validation_data = validation_set, shuffle=True)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 400, 512)          4689216   
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                110784    
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              66560     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

ValueError: Error when checking input: expected time_distributed_1_input to have 5 dimensions, but got array with shape (400, 64, 64, 3)

<h3 style="color:purple;">METHOD 2</h3>


In [13]:
from keras.layers.embeddings import Embedding

model = Sequential()
model.add(Embedding(46, 256, input_length=None))
model.add(LSTM(output_dim=128, activation='sigmoid', inner_activation='hard_sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop')
model.fit(train_set)
score = model.evaluate(X_test, Y_test, batch_size=16)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="sigmoid", units=128, recurrent_activation="hard_sigmoid")`
  """
/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1


ValueError: Error when checking input: expected embedding_1_input to have 2 dimensions, but got array with shape (400, 64, 64, 3)

<h3 style="color:purple;">METHOD 3</h3>

In [14]:
model=keras.models.Sequential()
import numpy as np
indices = np.array([64,64,3])
# shape = np.array([64,64,3])
shape = np.array([64,64,3])[indices.astype(int)]

shape = shape.reshape(1,shape)
model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(16, kernel_size=(3,3), data_format="channels_last"),input_shape=shape))
model.add(keras.layers.TimeDistributed(keras.layers.Activation("relu")))
model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(16, kernel_size =(3,3))))
model.add(keras.layers.TimeDistributed(keras.layers.Activation("relu")))
model.add(keras.layers.TimeDistributed(keras.layers.MaxPooling2D(pool_size=2)))
# model.add(keras.layers.TimeDistributed(keras.layers.Flatten()))
model.add(keras.layers.TimeDistributed(keras.layers.Reshape((280*38*16,))))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(32)))
model.add(keras.layers.GRU(512))
model.add(keras.layers.Dense(50))
model.add(keras.layers.Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])
history = model.fit(train_set, epochs=2, verbose=1, validation_data = validation_set, shuffle=True)

IndexError: index 64 is out of bounds for axis 0 with size 3

<h1> METHOD 4 </h1>

In [15]:
from keras.layers import Input, Embedding, GRU, LSTM, MaxPooling1D, GlobalMaxPool1D, CuDNNGRU
from keras.layers import Dropout, Dense, Activation, Flatten, Conv1D, SpatialDropout1D
from keras.models import Sequential
from sklearn.metrics import roc_auc_score

model = Sequential()
# model.add(Embedding(max_words, , input_length=max_len))
model.add(Conv1D(64,3, activation='relu',input_shape=(64, 64)))
model.add(SpatialDropout1D(0.2))
model.add(MaxPooling1D(4))
model.add(CuDNNGRU(64))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_set, epochs=5, batch_size=128, validation_data = validation_set)

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [16]:
test_loss , test_accuracy = cnn_model.evaluate(test_set)

35/35 [==============================] - 119s 3s/step


In [17]:
print("Test accuracy: "+ str(test_accuracy*100), "\nTest_loss: " + str(test_loss*100))

Test accuracy: 77.08695530891418 
Test_loss: 0.6793536711484194


In [18]:
from skimage.transform import resize
from PIL import Image
import numpy as np
im = Image.open(path+'/Test/character_28_la/11454.png')
im = np.array(im)
im = resize(im , (1,112,112,3))
y_pred = cnn_model.predict(im)


ValueError: Error when checking input: expected conv2d_1_input to have shape (64, 64, 3) but got array with shape (112, 112, 3)